![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/HuggingFace%20in%20Spark%20NLP%20-%20BertForZeroShotClassification.ipynb)

## Import BertForZeroShotClassification models from HuggingFace 🤗  into Spark NLP 🚀 

Let's keep in mind a few things before we start 😊 

- This feature is only in `Spark NLP 4.4.0` and after. So please make sure you have upgraded to the latest Spark NLP release
- You can import Bert models trained/fine-tuned for sequence classification via `BertForSequenceClassification` or `TFBertForSequenceClassification`. We can use these models for zero-shot classification.
  - These models are usually under `Sequence Classification` category and have `bert` in their labels
  - For zero-shot classification, we will use models trained on the nli data sets. The model should have been trained on the labels `contradiction`, `entailment` and `neutral`.
- Reference: [TFBertForSequenceClassification](https://huggingface.co/docs/transformers/main/en/model_doc/bert#transformers.TFBertForSequenceClassification)

## Export and Save HuggingFace model

- Let's install `HuggingFace` and `TensorFlow`. You don't need `TensorFlow` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock TensorFlow on `2.11.0` version and Transformers on `4.25.1`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.

In [1]:
!pip install -q transformers==4.25.1 tensorflow==2.11.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 63.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 76.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 59.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 65.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 82.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 kB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 107.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-datasets 4.9.2 requires protobuf>=3.20, but you have protobuf 3.19.6 which is incompatible.
tens

- HuggingFace comes with a native `saved_model` feature inside `save_pretrained` function for TensorFlow based models. We will use that to save it as TF `SavedModel`.
- We'll use [bert-base-mnli](https://huggingface.co/aloxatel/bert-base-mnli) model from HuggingFace as an example
- In addition to `TFBertForSequenceClassification` we also need to save the `BertTokenizer`. This is the same for every model, these are assets needed for tokenization inside Spark NLP.

In [3]:
from transformers import TFBertForSequenceClassification, BertTokenizer 
import tensorflow as tf

MODEL_NAME = 'aloxatel/bert-base-mnli'

tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)
tokenizer.save_pretrained('./{}_tokenizer/'.format(MODEL_NAME))

try:
  model = TFBertForSequenceClassification.from_pretrained(MODEL_NAME)
except:
  model = TFBertForSequenceClassification.from_pretrained(MODEL_NAME, from_pt=True)
    
# Define TF Signature
@tf.function(
  input_signature=[
      {
          "input_ids": tf.TensorSpec((None, None), tf.int32, name="input_ids"),
          "attention_mask": tf.TensorSpec((None, None), tf.int32, name="attention_mask"),
          "token_type_ids": tf.TensorSpec((None, None), tf.int32, name="token_type_ids"),
      }
  ]
)
def serving_fn(input):
    return model(input)

model.save_pretrained("./{}".format(MODEL_NAME), saved_model=True, signatures={"serving_default": serving_fn})

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

All the weights of TFBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


Let's have a look inside these two directories and see what we are dealing with:

In [4]:
!ls -l {MODEL_NAME}

total 427968
-rw-r--r-- 1 root root       813 Jun  6 15:13 config.json
drwxr-xr-x 3 root root      4096 Jun  6 15:13 saved_model
-rw-r--r-- 1 root root 438226204 Jun  6 15:13 tf_model.h5


In [5]:
!ls -l {MODEL_NAME}/saved_model/1

total 9208
drwxr-xr-x 2 root root    4096 Jun  6 15:13 assets
-rw-r--r-- 1 root root      56 Jun  6 15:13 fingerprint.pb
-rw-r--r-- 1 root root  166830 Jun  6 15:13 keras_metadata.pb
-rw-r--r-- 1 root root 9245668 Jun  6 15:13 saved_model.pb
drwxr-xr-x 2 root root    4096 Jun  6 15:13 variables


In [6]:
!ls -l {MODEL_NAME}_tokenizer

total 236
-rw-r--r-- 1 root root    125 Jun  6 15:12 special_tokens_map.json
-rw-r--r-- 1 root root    540 Jun  6 15:12 tokenizer_config.json
-rw-r--r-- 1 root root 231508 Jun  6 15:12 vocab.txt


- As you can see, we need the SavedModel from `saved_model/1/` path
- We also be needing `vocab.txt` from the tokenizer
- All we need is to just copy the `vocab.txt` to `saved_model/1/assets` which Spark NLP will look for
- In addition to vocabs, we also need `labels` and their `ids` which is saved inside the model's config. We will save this inside `labels.txt`

In [7]:
asset_path = '{}/saved_model/1/assets'.format(MODEL_NAME)

!cp {MODEL_NAME}_tokenizer/vocab.txt {asset_path}

In [8]:
# get label strings
labels = [model.config.id2label[l] for l, v in model.config.id2label.items()]
print(labels)

with open(asset_path+'/labels.txt', 'w') as f:
    f.write('\n'.join(labels))

['contradiction', 'entailment', 'neutral']


Voila! We have our `vocab.txt` and `labels.txt` inside assets directory

In [9]:
!ls -l {MODEL_NAME}/saved_model/1/assets

total 232
-rw-r--r-- 1 root root     32 Jun  6 15:14 labels.txt
-rw-r--r-- 1 root root 231508 Jun  6 15:14 vocab.txt


## Import and Save BertForZeroShotClassification in Spark NLP


- Let's install and setup Spark NLP in Google Colab
- This part is pretty easy via our simple script

In [17]:
! wget -q http://setup.johnsnowlabs.com/colab.sh -O - | bash

Installing PySpark 3.2.3 and Spark NLP 4.4.3
setup Colab for PySpark 3.2.3 and Spark NLP 4.4.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.5/281.5 MB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.8/489.8 kB 42.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 20.2 MB/s eta 0:00:00


Let's start Spark with Spark NLP included via our simple `start()` function

In [11]:
import sparknlp
# let's start Spark with Spark NLP
spark = sparknlp.start()

- Let's use `loadSavedModel` functon in `BertForZeroShotClassification` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `BertForZeroShotClassification` in runtime like `setMaxSentenceLength`, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.

In [12]:
from sparknlp.annotator import *
from sparknlp.base import *

zero_shot_classifier = BertForZeroShotClassification.loadSavedModel(
    '{}/saved_model/1'.format(MODEL_NAME),
    spark
    )\
    .setInputCols(["document", "token"]) \
    .setOutputCol("class") \
    .setCandidateLabels(["urgent", "mobile", "travel", "movie", "music", "sport", "weather", "technology"])

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [13]:
zero_shot_classifier.write().overwrite().save("./{}_spark_nlp".format(MODEL_NAME))

Let's clean up stuff we don't need anymore

In [ ]:
!rm -rf {MODEL_NAME}_tokenizer {MODEL_NAME}

Awesome 😎  !

This is your BertForZeroShotClassification model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀 

In [14]:
! ls -l {MODEL_NAME}_spark_nlp

total 436628
-rw-r--r-- 1 root root 447094331 Jun  6 15:16 bert_classification_tensorflow
drwxr-xr-x 5 root root      4096 Jun  6 15:16 fields
drwxr-xr-x 2 root root      4096 Jun  6 15:16 metadata


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny BertForSequenceClassification model 😊 

In [15]:
zero_shot_classifier_loaded = BertForZeroShotClassification.load("./{}_spark_nlp".format(MODEL_NAME))\
  .setInputCols(["document",'token'])\
  .setOutputCol("class")

This is how you can use your loaded classifier model in Spark NLP 🚀 pipeline:

In [16]:
from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline, PipelineModel

document_assembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

tokenizer = Tokenizer().setInputCols("document").setOutputCol("token")

pipeline = Pipeline(stages=[
    document_assembler,
    tokenizer,
    zero_shot_classifier_loaded
])

text = [["I have a problem with my iphone that needs to be resolved asap!!"],
        ["Last week I upgraded my iOS version and ever since then my phone has been overheating whenever I use your app."],
        ["I have a phone and I love it!"],
        ["I really want to visit Germany and I am planning to go there next year."],
        ["Let's watch some movies tonight! I am in the mood for a horror movie."],
        ["Have you watched the match yesterday? It was a great game!"],
        ["We need to harry up and get to the airport. We are going to miss our flight!"]]

# create a DataFrame in PySpark
inputDataset = spark.createDataFrame(text, ["text"])
model = pipeline.fit(inputDataset)
model.transform(inputDataset).select("class.result").show()

+------------+
|      result|
+------------+
|    [urgent]|
|[technology]|
|    [mobile]|
|    [travel]|
|     [movie]|
|     [sport]|
|    [urgent]|
+------------+



That's it! You can now go wild and use hundreds of `BertForSequenceClassification` models as zero-shot classifiers from HuggingFace 🤗 in Spark NLP 🚀 